In [1]:
using Statistics
using Distributions

epsilons = [0.0078125, 0.015625, 0.03125, 0.0625, 0.125, 0.25]
cs = [0.0625, 0.125, 0.25, 0.5, 1, 2, 4]
q0 = [0.25, 0.5, 1, 2, 4]
alphas = [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4]

8-element Array{Float64,1}:
 0.03125
 0.0625
 0.125
 0.25
 0.5
 1.0
 2.0
 4.0

In [2]:
numsims = 2000 ## necessary???
simlength = 200000
qlen = 10
## eps = 0.1

10

# Epsilon greedy

In [3]:
function epsilongreedy(q_s, simsteps, eps)
    ## initialize the Q and n variables
    Q = zeros(qlen)
    n = zeros(qlen)
    Rewards = zeros(simlength)

    for i in 1:simsteps
        ## epsilon greedy portion
        probrand = rand(1)[1]
        if probrand < eps
            A = rand(1:10)
        else
            _, A = findmax(Q)
        end
        ## calculate reward
        R = randn(1)[1] + q_s[A]
        ## update n and Q
        n[A] = n[A] + 1
        Q[A] = Q[A] + (1/n[A]) * (R - Q[A])
        Rewards[i] = R
        ## add noise, but only put in after we get normal one to run
        q_s = q_s + randn(10) * 0.01
    end
    return Rewards[trunc(Int, length(Rewards)/2 + 1):length(Rewards)]
end

epsilongreedy (generic function with 1 method)

In [4]:
avgRewEps = zeros(length(epsilons))
q_star = zeros(qlen)
for j in 1:length(epsilons), k in 1:numsims
    avgRewEps[j] = avgRewEps[j] + mean(epsilongreedy(q_star, simlength, epsilons[j]))
end
avgRewEps / numsims

6-element Array{Float64,1}:
 4.580907496514873
 4.4834352223576
 4.479404372827818
 4.411259142292989
 4.131025008845298
 3.6414797764542706

In [5]:
## 12.81 seconds, on my timing.  Multiply times 100 to get full run???

# Upper Confidence Bound

In [6]:
function uppconfbd(q_s, simsteps, c)
    ## initialize the Q and n variables
    Q = zeros(qlen)
    n = zeros(qlen)
    Rewards = zeros(simlength)
    
    for i in 1:qlen
        ## first qlen steps are just running through all options
        R = randn(1)[1] + q_s[i]
        n[i] = n[i] + 1
        Q[i] = Q[i] + (1/n[i]) * (R - Q[i])
        Rewards[i] = R
    end
    ## now for the rest of the steps
    for i in (qlen+1):simsteps
        Qplus = Q + c * sqrt(log(i)) * (1 / sqrt.(log.(n)))'
        _, A = findmax(Qplus)
        ## calculate reward
        R = randn(1)[1] + q_s[A]
        ## update n and Q
        n[A] = n[A] + 1
        Q[A] = Q[A] + (1/n[A]) * (R - Q[A])
        Rewards[i] = R
        ## add noise, but only put in after we get normal one to run
        q_s = q_s + randn(10) * 0.01
    end
    return Rewards[trunc(Int, length(Rewards)/2 + 1):length(Rewards)]
end

uppconfbd (generic function with 1 method)

In [7]:
avgRewUCB = zeros(length(cs))
q_star = zeros(qlen)
for j in 1:length(cs), k in 1:numsims
    avgRewUCB[j] = avgRewUCB[j] + mean(uppconfbd(q_star, simlength, cs[j]))
end
avgRewUCB / numsims

7-element Array{Float64,1}:
 2.5809831382024777
 2.501275901898166
 2.5657822151075247
 2.5129103220800295
 2.3295224477026015
 1.973619806068027
 0.8783037327602881

In [8]:
## 42.62 seconds, multiply by 100?

# Optimistic Initial Value

In [9]:
function optinitval(q_s, simsteps, Q0)
    ## initialize the Q and n variables
    Q = ones(qlen) * Q0
    Rewards = zeros(simlength)

    for i in 1:simsteps
        ## epsilon greedy portion
        _, A = findmax(Q)
        R = randn(1)[1] + q_s[A]
        ## update Q
        Q[A] = Q[A] + 0.1 * (R - Q[A])
        Rewards[i] = R
        ## add noise, but only put in after we get normal one to run
        q_s = q_s + randn(10) * 0.01
    end
    return Rewards[trunc(Int, length(Rewards)/2 + 1):length(Rewards)]
end

optinitval (generic function with 1 method)

In [10]:
avgRewOIV = zeros(length(q0))
q_star = zeros(qlen)
for j in 1:length(q0), k in 1:numsims
    avgRewOIV[j] = avgRewOIV[j] + mean(optinitval(q_star, simlength, q0[j]))
end
avgRewOIV / numsims

5-element Array{Float64,1}:
 4.381110341447758
 4.439002961200888
 4.427588459387116
 4.374878246927533
 4.304071235455939

In [11]:
## 9.12 seconds

# Gradient Bandit Algorithm

In [12]:
function gradband(q_s, simsteps, alpha)
    ## initialize the H, Rbar, probs variables
    H = zeros(qlen)
    probs = ones(qlen) * (1 / qlen)
    Rbar = 0
    Rewards = zeros(simlength)

    for i in 1:simsteps
        ## epsilon greedy portion
        test = Multinomial(1, probs)
        A = findmax(rand(test, 1))[2][1] ## Cartesian index workaround
        R = randn(1)[1] + q_s[A]
        Rbar = Rbar + (1 / i) * (R - Rbar)
        ## update H
        temp = H[A]
        H = H - alpha * (R - Rbar) * probs
        H[A] = temp + alpha * (R - Rbar) * (1 - probs[A])
        Rewards[i] = R
        ## updates probs
        probs = exp.(H) / sum(exp.(H))
        ## add noise, but only put in after we get normal one to run
        q_s = q_s + randn(10) * 0.01
    end
    return Rewards[trunc(Int, length(Rewards)/2 + 1):length(Rewards)]
end

gradband (generic function with 1 method)

In [13]:
avgRewGBA = zeros(length(alphas))
q_star = zeros(qlen)
for j in 1:length(alphas), k in 1:numsims
    avgRewGBA[j] = avgRewGBA[j] + mean(gradband(q_star, simlength, alphas[j]))
end
avgRewGBA / numsims

8-element Array{Float64,1}:
 3.771695631589715
 3.6596456445785663
 3.5376636249745554
 3.392754616377856
 2.962147441765465
 2.5838196873396795
 1.95240147285649
 1.3989306596608762

In [14]:
## 58.96 seconds

In [15]:
58.96 + 9.12 + 42.62 + 12.81

123.50999999999999

In [16]:
123.51 * 100 / 60^2

3.430833333333333

In [ ]:
## sim results
avgRewEpssim = [4.5809, 4.4834, 4.4794, 4.4112, 4.1310, 3.6414]
avgRewUCBsim = [2.5809, 2.5012, 2.5657, 2.5129, 2.3295, 1.9736, 0.8783]
avgRewOIVsim = [4.3811, 4.4390, 4.4275, 4.3748, 4.3040]
avgRewGBAsim = [3.7716, 3.6596, 3.5376, 3.3927, 2.9621, 2.5838, 1.9524, 1.3989]